In [6]:
from sklearn.metrics import r2_score
import numpy as np
from pymoo.factory import get_problem, get_visualization
from scipy.stats import qmc
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from pymoo.core.problem import Problem
from scipy.stats import norm

#TODO: run GA with current surrogate and then compute EI from solutions generated by GA

class ExpectedImprovement(Problem):
    def __init__(self, p, model, fxb):
        """
        p: problem object
        model: model object
        fxb: best objective function value found
        """
        super().__init__(n_var=p.n_var, n_obj=1,
                         n_constr=0, xl=p.xl, xu=p.xu)
        self.model = model
        self.fxb = fxb

    def _evaluate(self, X, out, *args, **kwargs):
        x_mean, x_std = self.model.predict(X, return_std=True)
        x_var = np.sqrt(x_std).reshape(-1, 1)
        epsilon = 1e-5
        # expected improvement
        inner = (self.fxb - x_mean - epsilon)/x_var
        left = (self.fxb - x_mean - epsilon) * norm.cdf(inner)
        right = x_var * norm.pdf(inner)
        EI = left + right
        # negate to fit the interface requiring minimization
        out["F"] = -EI

class Surrogate(Problem):
    """
    Surrogate to Problem wrapper
    """
    def __init__(self, p, model):
        """
        p: problem object
        """
        super().__init__(n_var=p.n_var, n_obj=1,
                         n_constr=0, xl=p.xl, xu=p.xu)
        self.model = model

    def _evaluate(self, X, out, *args, **kwargs):
        out["F"] = self.model.predict(X)


def optimize():
    """
    use EI and the mean prediction to solve any single objective benchmark problem
    """
    seed = 1
    max_eval = 50
    # problem = get_problem("ackley", n_var=1, a=20, b=1/5, c=2 * np.pi)
    problem = get_problem("rosenbrock", n_var=2)
    model = GaussianProcessRegressor()
    
    optimizer = GA(pop_size=100)
    optimizer_acquisition = GA(pop_size=100)

    #generate initial sample
    lhs = qmc.LatinHypercube(problem.n_var, seed=seed)
    X = lhs.random(50)
    X = qmc.scale(X, problem.xl, problem.xu)
    y = problem.evaluate(X)

    #Build surrogates with initial data
    model.fit(X, y)

    surrogateProblem = Surrogate(problem, model)
    for _ in range(max_eval):
        #find optimum of surrogate
        best = minimize(surrogateProblem, optimizer, ("n_gen", 10), seed=seed)
        #construct EI problem with found optimum
        ei = ExpectedImprovement(problem, model, best.F)

        #optimize considering expected improvement
        res = minimize(ei, optimizer_acquisition, ("n_gen", 10), seed=seed)
        x_t = res.X
        #print(res.X, -res.F, end = ", ")
        #evaluate the solution that maximizes expected improvement
        #with real function
        y_t = problem.evaluate(x_t)

        #add newly evaluated to data
        # X = np.vstack((X, x_t))
        # y = np.vstack((y, y_t))
        X = np.append(X, [x_t], axis=0)
        y = np.append(y, [y_t], axis=0)
        #Rebuild the surrogates with the new data
        model.fit(X, y)

    return best.X, best.F

sol = optimize()
print("sol", sol)


sol (array([1.05692369, 1.1119617 ]), array([0.03909378]))
